In [2]:
#Abertura de demanda 

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from IPython.display import Image, display  # Para exibir imagens no notebook


# Configurações do servidor SMTP
smtp_server = "smtp.gmail.com"
smtp_port = 587

# Suas credenciais
email = "vitoracavaco40@gmail.com"  # Substitua pelo seu e-mail
senha = ""  # Substitua pela sua senha (ou senha de app, se tiver 2FA ativado)

# Caminho para o chromedriver
driver_path = r"C:\Users\Vitor\Documents\chromedriver-win64\chromedriver.exe"

# Crie uma instância de Service passando o caminho do driver
service = Service(driver_path)

# Inicie o ChromeDriver com o serviço e as opções de download
driver = webdriver.Chrome(service=service)

print("Robo iniciando pesquisas......\n")

# Acessa o site
driver.get("https://lista.mercadolivre.com.br/celulares-telefones/celulares-smartphones/iphone/linha-iphone-16/iphone-16/novo/iphone-16_NoIndex_True#applied_filter_id%3DITEM_CONDITION%26applied_filter_name%3DCondi%C3%A7%C3%A3o%26applied_filter_order%3D4%26applied_value_id%3D2230284%26applied_value_name%3DNovo%26applied_value_order%3D1%26applied_value_results%3D17%26is_custom%3Dfalse")

driver.maximize_window()

wait = WebDriverWait(driver, 40)

array_preco_ml = []

array_link_ml = []

array_imagem_ml = []

array_preco_amz = []

array_link_amz = []

array_imagem_amz = []

lista_iphone_ml = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//a[contains(@class, "poly-component__title")]')))

preco_iphone_ml  = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//div[contains(@class, "poly-price__current")]//span[contains(@class, "andes-money-amount__fraction")]')))

imagem_iphone_ml = wait.until(EC.visibility_of_all_elements_located((By.XPATH,'//div[contains(@class, "poly-card__portada")]//img[contains(@class, "poly-component__picture")]')))

link_iphone_ml  = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//h3[contains(@class, "poly-component__title-wrapper")]//a[contains(@class, "poly-component__title")]')))

print("--------- Mercado Livre ---------\n")

for valor, lista in enumerate(lista_iphone_ml):

    texto = lista.text

    preco = float(preco_iphone_ml[valor].text)

    driver.execute_script("arguments[0].scrollIntoView(true);", imagem_iphone_ml[valor])

    imagem = imagem_iphone_ml[valor].get_attribute("src")

    link  = link_iphone_ml[valor].get_attribute("href")

    print(f"{texto}  - R$: {preco} - Link: {link} ")
    #print(f"{texto}  - R$: {preco} - url: {imagem}")
    display(Image(url=imagem, width=150, height=150))
    print("\n")

    if (preco == 5.698):

        array_preco_ml.append(preco)
        array_link_ml.append(link)
        array_imagem_ml.append(imagem)

print('\n-----------------------------------\n')

driver.get("https://www.amazon.com.br/ref=nav_logo")

pesquisa_amazon = wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="twotabsearchtextbox"]')))
pesquisa_amazon.send_keys("Iphone 16")
pesquisa_amazon.send_keys(Keys.RETURN)

lista_iphone_amz = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//h2[contains(@class, "a-size-base-plus a-spacing-none a-color-base a-text-normal")]')))

preco_iphone_amz = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//span[contains(@class, "a-price-whole")]')))

imagem_iphone_amz = wait.until(EC.visibility_of_all_elements_located((By.XPATH, '//div[contains(@class, "a-section aok-relative s-image-square-aspect")]//img[contains(@class,"s-image")]')))

link_iphone_amz = wait.until(EC.visibility_of_all_elements_located((By.XPATH, '//a[contains(@class, "a-link-normal s-line-clamp-4 s-link-style a-text-normal")]')))

print("--------- Amazon ---------\n")

for i in range (8):
 
    texto = (lista_iphone_amz[i].text).upper()

    if "IPHONE 16" in texto:

        preco = float(preco_iphone_amz[i].text)

        imagem = imagem_iphone_amz[i].get_attribute("src")

        link  = link_iphone_amz[i].get_attribute("href")

        print(f"{texto}  - R$: {preco} - Link: {link} \n")
        
        display(Image(url=imagem, width=150, height=150))

        print("\n")

        if (preco == 5.698):

            array_preco_amz.append(preco)
            array_link_amz.append(link)
            array_imagem_amz.append(imagem)

driver.quit()


if array_preco_ml:

  # Criação da mensagem
    msg = MIMEMultipart()
    msg['From'] = email
    msg['To'] = "vitoracavaco40@gmail.com"  # Substitua pelo e-mail do destinatário
    msg['Subject'] = "Promoção Iphone"

    #valores = ", ".join(map(str,array_preco_ml))
    
    # Corpo do e-mail

    corpo_email = f"Promoção do Iphone 16 no mercado livre\n\n"

    for preco, link, imagem in zip(array_preco_ml, array_link_ml, array_imagem_ml):
        corpo_email += f"- Preço: R$ {preco}\n\n  Link: {link}\n\n  Imagem: {imagem}\n\n"

    msg.attach(MIMEText(corpo_email, 'plain'))

    # Conectar ao servidor SMTP e enviar o e-mail
    try:
        # Estabelecendo conexão com o servidor SMsTP
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()  # Ativa a criptografia TLS para segurança

        # Login na conta
        server.login(email, senha)

        # Enviar o e-mail
        server.sendmail(msg['From'], msg['To'], msg.as_string())

        print("E-mail enviado com sucesso!")

    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

    finally:
        # Fechar a conexão
        server.quit()

if array_preco_amz:
    
  # Criação da mensagem
    msg = MIMEMultipart()
    msg['From'] = email
    msg['To'] = "vitoracavaco40@gmail.com"  # Substitua pelo e-mail do destinatário
    msg['Subject'] = "Promoção Iphone"

    #valores = ", ".join(map(str, array_preco_amz))

    # Corpo do e-mail
    corpo_email = f"Promoção do Iphone 16 na amazon\n\n"

    for preco, link, imagem in zip(array_preco_amz, array_link_amz, array_imagem_amz):
        corpo_email += f"- Preço: R$ {preco}\n\n  Link: {link}\n\n  Imagem: {imagem}\n\n"

    msg.attach(MIMEText(corpo_email, 'plain'))

    # Conectar ao servidor SMTP e enviar o e-mail
    try:
        # Estabelecendo conexão com o servidor SMTP
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()  # Ativa a criptografia TLS para segurança

        # Login na conta
        server.login(email, senha)

        # Enviar o e-mail
        server.sendmail(msg['From'], msg['To'], msg.as_string())

        print("E-mail enviado com sucesso!")

    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

    finally:
        # Fechar a conexão
        server.quit()        


Robo iniciando pesquisas......

--------- Mercado Livre ---------

Capa Couro Para Capa Tipo Carteira Anti-queda Para iPhone  - R$: 113.0 - Link: https://click1.mercadolivre.com.br/mclics/clicks/external/MLB/count?a=KCodNRNrYKujvXgnnvk5uOiqSqBLcOz%2FWanwXb1dhoxnMDPSXfg8TItMz%2FbJ8vl%2BLSuSasRS5pIE%2FaPRmw2l4dIlr12w6PnvnH1eegaMob45a8FuEVtbJTIBAQY4taqjs0nrCMi%2Fu0XUOyA%2FSAMqhzTBYckUEbHnKohqyvxeOUPksrFUtP1SAt8liNZ%2F6XpHiu%2B1QyMyioPsIt5APEtDCaZd6CtbK1OiWrINkDwIJKOnJCfgTeVMxrMYeQWEBnpjdFZ7sRpjxTAOFPt1qGYmQVoZzYrdBPTSHwUjEWfWqTJOAw%2FN8mSvOZ9kXGxzltt6REsedwvHDGIx2KK64nILJBgsvutif2ifFA6nizwZ%2BfsO5VH9McNCU60DBsd%2BRg1CKA%2BGIG01UH9iMZRBjQuPC2gSHFQJK%2BXjmtEGeH4ynr44Zym%2BT3rPfwDGKpZb1T4rJkbcnBg%2F3BnP9OcqDKCjCNlAve86aX3TF34hMtRzrcOL99iGl68iGKQ2L9IWSKQGD%2BAtouS3Ngdmbz8eTHyqBDckOJ9aTPZftDnhHiMcjPe4vZ2fMvyFoywQx%2FySnRfVnjPR8sEXJitUazaIcTnBrZtwmzezuaOTF0%2F0q4t5qcXbE2Iph%2Bfi4iD8U7hpwHNXmAT1WSCVvZ1iHNQJZBFjqnkWYtWfOvf3ywE65QXQEf0OqJDdYHwpSDLnAzZteTUNImGEmiwM84TMuqcvYf3fqC2L7Q82VlmE6kqptuJVWZ



Suporte De Saco De Cartão Zfold6 Para Celular Samsung  - R$: 155.0 - Link: https://click1.mercadolivre.com.br/mclics/clicks/external/MLB/count?a=PAsibn73rSRG6SWtzp4%2BsEU%2FqO27EzYMJiVb6btDcH286k%2FB5DB36TQFtOFKfbmD1sQ89EhuGB8388cbSmjh6Dpkf4kOtGqGj%2Bgm637XcMB%2Bie2oibHSj2%2FpO6GugzLINZDGuIPA%2Bc%2FJ3z4AEvfzfTe4ZHWnirhcxLGl%2BZ62IgTE6vyGa7rnEfc90v2MOXHCwxHDJZABRadVxhf53Ss%2BoNIHWglq52uXW7hs%2F7pdB9MKJvOHMpZhmYOJnM6OeqSlzDo1UTdQjB2BwfWCNOWefUSD4D3G7PrvbTIDRHpsrZ67kkFWH5Gtz%2BJGT6dtufpVjUURjC3TOH9NUCrsA6KFAf977ulQYDYW5OML8ZFJvwUaxxlRY0JcVZTiPTLvVL05d%2B2RtZCeVmw2uwfLHK%2FNyngAukpn4oi9m4AF7ca5HsT40k%2Bwp8yR%2FVs%2BJYgS1H7iAn3HfMklJh%2BtJAqFk1YmqVkFDKxg4KT2uOFZF4Jn4qtvd1oeHWAK2NVrjfM7pe1g39VzRZKoWwJmknxkiTHpBWZe20%2BfdFrYTLHHw2UVeZWN8rAZjEn2KmXjgaYchNg5ve8pxfhG2elqvaPINZs4GPKcSuFvFy%2BZSFdPUHUS8nW9xSkTzECy2ue%2FAU5AaT2qyqtVDevixWkkyaxiapGVommsd3QdVtNrsvjZ%2BM93OcKCGWU6HYLXM9xjck7oSP80zbicoW%2BN40RD7EPLRGE8Gizvl95ppzYuJKCNVLB4by5I40trczf2cLCXg%2BsBU1jtJseWbNBJl0xJYchveihuGI%2BdY6MDayxI9C4



Apple iPhone 16 (128 GB) - Preto - Distribuidor Autorizado  - R$: 5.698 - Link: https://www.mercadolivre.com.br/apple-iphone-16-128-gb-preto-distribuidor-autorizado/p/MLB1040287808#polycard_client=search-nordic&searchVariation=MLB1040287808&wid=MLB5120546360&position=4&search_layout=stack&type=product&tracking_id=18ab4ec8-49e4-4f41-9239-68a7b3eead1c&sid=search 




Apple iPhone 16 (128 GB) - Rosa - Distribuidor Autorizado  - R$: 5.698 - Link: https://www.mercadolivre.com.br/apple-iphone-16-128-gb-rosa-distribuidor-autorizado/p/MLB1040287802#polycard_client=search-nordic&searchVariation=MLB1040287802&wid=MLB5120624320&position=5&search_layout=stack&type=product&tracking_id=18ab4ec8-49e4-4f41-9239-68a7b3eead1c&sid=search 




Apple iPhone 16 (128 GB) - Branco - Distribuidor Autorizado  - R$: 5.698 - Link: https://www.mercadolivre.com.br/apple-iphone-16-128-gb-branco-distribuidor-autorizado/p/MLB1040287791#polycard_client=search-nordic&searchVariation=MLB1040287791&wid=MLB5120584956&position=6&search_layout=stack&type=product&tracking_id=18ab4ec8-49e4-4f41-9239-68a7b3eead1c&sid=search 


...



iPhone 16 256gb Novo Lacrado  - R$: 7.099 - Link: https://produto.mercadolivre.com.br/MLB-5265617794-iphone-16-256gb-novo-lacrado-_JM?searchVariation=182800173184#polycard_client=search-nordic&searchVariation=182800173184&position=28&search_layout=stack&type=item&tracking_id=18ab4ec8-49e4-4f41-9239-68a7b3eead1c 




Apple iPhone 16 128g Lacrado 1 Ano De Garantia Novo Tipo C  - R$: 6.699 - Link: https://produto.mercadolivre.com.br/MLB-3961581249-apple-iphone-16-128g-lacrado-1-ano-de-garantia-novo-tipo-c-_JM?searchVariation=182814722920#polycard_client=search-nordic&searchVariation=182814722920&position=29&search_layout=stack&type=item&tracking_id=18ab4ec8-49e4-4f41-9239-68a7b3eead1c 




Apple iPhone 16 (128 GB) - Branco  - R$: 8.59 - Link: https://www.mercadolivre.com.br/apple-iphone-16-128-gb-branco/p/MLB40287791#polycard_client=search-nordic&searchVariation=MLB40287791&wid=MLB5266537536&position=3&search_layout=stack&type=product&tracking_id=18ab4ec8-49e4-4f41-9239-68a7b3eead1c&sid=search 





-----------------------------------



NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=132.0.6834.111)
Stacktrace:
	GetHandleVerifier [0x00007FF6B172CC05+28741]
	(No symbol) [0x00007FF6B168FED0]
	(No symbol) [0x00007FF6B1524FBA]
	(No symbol) [0x00007FF6B14FED55]
	(No symbol) [0x00007FF6B15A6DE7]
	(No symbol) [0x00007FF6B15BF8B2]
	(No symbol) [0x00007FF6B159FBD3]
	(No symbol) [0x00007FF6B1569F43]
	(No symbol) [0x00007FF6B156B2D1]
	GetHandleVerifier [0x00007FF6B1A5DE3D+3375741]
	GetHandleVerifier [0x00007FF6B1A7096A+3452330]
	GetHandleVerifier [0x00007FF6B1A6671D+3410781]
	GetHandleVerifier [0x00007FF6B17F653B+854395]
	(No symbol) [0x00007FF6B169BE0F]
	(No symbol) [0x00007FF6B1697714]
	(No symbol) [0x00007FF6B16978AD]
	(No symbol) [0x00007FF6B1686189]
	BaseThreadInitThunk [0x00007FFB9336E8D7+23]
	RtlUserThreadStart [0x00007FFB9477FBCC+44]
